In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import os

# --- CONFIGURATION ---
FILE_PATH = r"C:\Users\wolfr\OneDrive\Desktop\Trades01072026.csv"
OUTPUT_NAME = "Trading_Quant_Dashboard.html"

def generate_dashboard():
    if not os.path.exists(FILE_PATH):
        print(f"Error: Could not find the file at {FILE_PATH}")
        return

    # --- DATA PROCESSING ---
    df = pd.read_csv(FILE_PATH)
    
    # Explicitly set format to remove the UserWarning
    df['Date/Time'] = pd.to_datetime(df['Date/Time'], format='%m/%d/%Y %I:%M:%S %p')
    df = df.sort_values('Date/Time')

    # Calculations
    df['Cumulative P/L'] = df['Net P/L'].cumsum()
    df['Peak'] = df['Cumulative P/L'].cummax()
    df['Drawdown'] = df['Cumulative P/L'] - df['Peak']
    df['Hour'] = df['Date/Time'].dt.hour
    
    # Performance Stats
    total_net_pl = df['Net P/L'].sum()
    total_trades = len(df)
    wins = df[df['Net P/L'] > 0]
    losses = df[df['Net P/L'] < 0]
    win_rate = (len(wins) / total_trades) * 100 if total_trades > 0 else 0
    profit_factor = (wins['Net P/L'].sum() / abs(losses['Net P/L'].sum())) if len(losses) > 0 else np.inf
    avg_win = wins['Net P/L'].mean() if not wins.empty else 0
    avg_loss = losses['Net P/L'].mean() if not losses.empty else 0
    expectancy = (win_rate/100 * avg_win) + ((1 - win_rate/100) * avg_loss)

    # --- VISUALS ---
    # 1. Equity Curve
    fig_equity = px.line(df, x='Date/Time', y='Cumulative P/L', title="Cumulative Net P/L")
    fig_equity.update_traces(line_color='#00d2ff', fill='tozeroy')

    # 2. Drawdown (QuantStats Style)
    fig_dd = px.area(df, x='Date/Time', y='Drawdown', title="Underwater / Drawdown Profile")
    fig_dd.update_traces(line_color='#ff4b4b', fillcolor='rgba(255, 75, 75, 0.3)')

    # 3. P/L by Side (Buy vs Sell)
    side_pl = df.groupby('Side')['Net P/L'].sum().reset_index()
    fig_side = px.bar(side_pl, x='Side', y='Net P/L', color='Side', 
                      color_discrete_map={'Buy': '#39d353', 'Sell': '#58a6ff'}, title="Buy vs Sell Performance")

    # 4. Hourly Performance
    hourly_pl = df.groupby('Hour')['Net P/L'].sum().reset_index()
    fig_hour = px.bar(hourly_pl, x='Hour', y='Net P/L', title="Profitability by Hour",
                      color='Net P/L', color_continuous_scale='RdYlGn')

    # Apply Dark Theme to all
    for f in [fig_equity, fig_dd, fig_side, fig_hour]:
        f.update_layout(template="plotly_dark", margin=dict(l=20, r=20, t=50, b=20))

    # --- HTML GENERATION ---
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Quant Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ background: #0d1117; color: #c9d1d9; font-family: sans-serif; padding: 20px; }}
            .grid {{ display: grid; grid-template-columns: 1fr 1fr; gap: 20px; }}
            .card {{ background: #161b22; border: 1px solid #30363d; border-radius: 8px; padding: 15px; margin-bottom: 20px; }}
            .stat-val {{ font-size: 24px; font-weight: bold; color: #58a6ff; }}
            .header {{ display: flex; justify-content: space-between; align-items: center; margin-bottom: 30px; }}
            .pos {{ color: #39d353; }} .neg {{ color: #f85149; }}
        </style>
    </head>
    <body>
        <div class="header card">
            <div><h1>Trading Session Report</h1><p>{df['Date/Time'].dt.date.iloc[0]}</p></div>
            <div style="text-align:right">
                <div class="stat-val pos">${total_net_pl:,.2f}</div>
                <div style="font-size:12px; color:#8b949e">TOTAL NET P/L</div>
            </div>
        </div>

        <div class="grid">
            <div class="card">Win Rate: <span class="pos">{win_rate:.1f}%</span></div>
            <div class="card">Profit Factor: <span class="stat-val" style="color:white">{profit_factor:.2f}</span></div>
            <div class="card">Total Trades: <span class="stat-val" style="color:white">{total_trades}</span></div>
            <div class="card">Expectancy: <span class="pos">${expectancy:.2f}</span></div>
        </div>

        <div class="grid">
            <div class="card">{fig_equity.to_html(full_html=False, include_plotlyjs=False)}</div>
            <div class="card">{fig_dd.to_html(full_html=False, include_plotlyjs=False)}</div>
            <div class="card">{fig_side.to_html(full_html=False, include_plotlyjs=False)}</div>
            <div class="card">{fig_hour.to_html(full_html=False, include_plotlyjs=False)}</div>
        </div>
    </body>
    </html>
    """

    with open(OUTPUT_NAME, "w", encoding="utf-8") as f:
        f.write(html_content)
    
    print(f"Success! Dashboard created at: {OUTPUT_NAME}")
    os.startfile(OUTPUT_NAME)

if __name__ == "__main__":
    generate_dashboard()

Success! Dashboard created at: Trading_Quant_Dashboard.html
